In [3]:
import urllib
import os
import zipfile
import pandas as pd
import tensorflow as tf

In [4]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data

In [5]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if logs.get('mae') < 0.1 and logs.get('val_mae') < 0.1:
      print('mae bellow 0.1, cancel training!')
      self.model.stop_training=True

In [6]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(n_past+n_future, shift=shift, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(n_past+n_future))
    dataset = dataset.map(lambda window: (window[:n_past], window[n_past:]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [7]:
df = pd.read_csv('household_power_consumption.csv', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)

<ipython-input-7-0a7171dcd7ba>:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('household_power_consumption.csv', sep=',',


In [30]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [32]:
df.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000,86400.000000
mean,1.644244,0.128601,240.964285,6.952023,1.305127,1.878669,7.514213
std,1.335542,0.117621,3.498536,5.629463,6.682567,7.567679,8.671909
min,0.194000,0.000000,224.680000,0.800000,0.000000,0.000000,0.000000
25%,0.396000,0.000000,238.610000,1.800000,0.000000,0.000000,0.000000
50%,1.416000,0.116000,241.220000,5.800000,0.000000,0.000000,0.000000
75%,2.414000,0.196000,243.470000,10.000000,0.000000,1.000000,17.000000
max,9.272000,0.874000,251.700000,40.400000,77.000000,78.000000,20.000000


In [8]:
N_FEATURES = len(df.columns)

# Normalizes the data
data = df.values
split_time = int(len(data) * 0.5)
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

# Splits the data into training and validation sets.
x_train = data[:split_time]
x_valid = data[split_time:]

BATCH_SIZE = 32
N_PAST = 24 # Number of past time steps based on which future observations should be predicted
N_FUTURE = 24  # Number of future time steps which are to be predicted.
SHIFT = 1  # By how many positions the window slides to create a new window of observations.

In [9]:
train_set = windowed_dataset(x_train, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT)
valid_set = windowed_dataset(x_valid, BATCH_SIZE, N_PAST, N_FUTURE, SHIFT)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(N_PAST+N_FUTURE, 5, activation='relu', input_shape=[N_PAST, N_FEATURES]),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(N_FUTURE*N_FEATURES),
    tf.keras.layers.Reshape((N_FUTURE, N_FEATURES)),
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 20, 48)            1728      
                                                                 
 lstm (LSTM)                 (None, 20, 32)            10368     
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 20)                660       
                                                                 
 dense_1 (Dense)             (None, 168)               3528      
                                                                 
 reshape (Reshape)           (None, 24, 7)             0         
                                                                 
Total params: 24604 (96.11 KB)
Trainable params: 24604 (

In [12]:
model.compile(
    loss='huber',
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9),
    metrics=['mae']
)

In [13]:
callback = myCallback()

model.fit(
    train_set,
    epochs=100,
    validation_data=valid_set,
    callbacks=[callback]
)

Epoch 1/100
1349/1349 [==============================] - 32s 19ms/step - loss: 0.0324 - mae: 0.1357 - val_loss: 0.0314 - val_mae: 0.1324
Epoch 2/100
1349/1349 [==============================] - 35s 26ms/step - loss: 0.0299 - mae: 0.1345 - val_loss: 0.0291 - val_mae: 0.1315
Epoch 3/100
1349/1349 [==============================] - 23s 17ms/step - loss: 0.0278 - mae: 0.1334 - val_loss: 0.0271 - val_mae: 0.1304
Epoch 4/100
1349/1349 [==============================] - 24s 18ms/step - loss: 0.0260 - mae: 0.1321 - val_loss: 0.0254 - val_mae: 0.1291
Epoch 5/100
1349/1349 [==============================] - 45s 34ms/step - loss: 0.0244 - mae: 0.1308 - val_loss: 0.0238 - val_mae: 0.1279
Epoch 6/100
1349/1349 [==============================] - 38s 28ms/step - loss: 0.0230 - mae: 0.1294 - val_loss: 0.0224 - val_mae: 0.1265
Epoch 7/100
1349/1349 [==============================] - 30s 22ms/step - loss: 0.0216 - mae: 0.1279 - val_loss: 0.0211 - val_mae: 0.1251
Epoch 8/100
1349/1349 [==================